# Test sequences for pipeline
---
### About the data

Fasta files were retrieved from Griffith's lab tutorial on RNA-seq https://github.com/griffithlab/rnaseq_tutorial:

`Malachi Griffith*, Jason R. Walker, Nicholas C. Spies, Benjamin J. Ainscough, Obi L. Griffith*. 2015. Informatics for RNA-seq: A web resource for analysis on the cloud. PLoS Comp Biol. 11(8):e1004393.`

*The practice dataset includes 3 replicates of data from the HCC1395 breast cancer cell line and 3 replicates of data from HCC1395BL matched lymphoblastoid line. So, this will be a tumor vs normal (cell line) comparison. The reads are paired-end 151-mers generated on an Illumina HiSeq instrument. The test data has been pre-filtered for reads that appear to map to chromosome 22.*

In [1]:
import sys
import os
from os import listdir,path
import pandas as pd
import io
import subprocess
import json
from collections import OrderedDict

sequences_url = r'http://genomedata.org/rnaseq-tutorial/practical.tar'

output_path = path.abspath('test_data')
output_config_file = path.join(output_path,'config.json')
output_fasta_path = path.join(output_path,'fastq')
output_experimental_design = path.join(output_path,'experimental_design.tsv')
annotation_gtf = path.join(output_path,'gencode','gencode.v34.annotation.gtf.gz')
reference_fasta = path.join(output_path,'gencode','gencode.v34.transcripts.fa.gz')

---
# <font color="red">SKIP</font>
### Download fasta files

In [2]:
%%bash -s "$sequences_url" "$output_path"
mkdir -p "$2/fastq"
wget "$1" -P "$2/"
tar -C "$2/fastq/" -xvf "$2/practical.tar"
rm "$2/practical.tar"

Process is interrupted.


### Download gencode files

In [ ]:
%%bash -s "$output_path"
gencode_path="$1/gencode"
mkdir -p "$gencode_path"
cd "$gencode_path"
wget "ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/gencode.v34.annotation.gtf.gz"
wget "ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/gencode.v34.transcripts.fa.gz"

---
# <font color="green">GO</font>
### Create template for configuration files

In [3]:
def create_config_stub(output_config_file,fastq_path,experimental_design,annotation_gtf,reference_fasta,
                       project_name='test',group='',email='',                       
                       extension_fastq='fastq.gz',delim_fastq_name='_',suffix_mate1='R1',suffix_mate2='R2',
                       star_reference='',star_threads='8',star_RAM='40000000000',star_sort_RAM='40000000000',
                       sort_mem='16G',samtools_sortByName_threads='4',
                       conda_path=None,env_name=None,generate_design=False,design_condition_parser=None):
    data = OrderedDict()
    data['project_name']=project_name
    data['group']=group
    data['email']=email
    data['fastqFolder']=fastq_path if fastq_path[-1]=='/' else fastq_path+'/'
    data['extension']=extension_fastq
    data['delim']=delim_fastq_name
    data['mates']={'mate1':suffix_mate1,'mate2':suffix_mate2}
    data['gtf']=annotation_gtf
    data['reference_fasta']=reference_fasta
    data['experimental_design']=experimental_design
    #~~~STAR
    data['star_reference']=star_reference
    data['star_threads']=star_threads
    data['star_RAM']=star_RAM
    data['star_sort_RAM']=star_sort_RAM
    #~~~SAMTOOLS
    data['sort_mem']=sort_mem
    data['samtools_sortByName_threads']=samtools_sortByName_threads
    
    #~~~Get from conda env
    if env_name is not None and conda_path is not None:        
        call = '%s list -n %s'%(path.join(conda_path,'bin','conda'),env_name)        
        s = subprocess.check_output(call,shell=True).decode()        
        df = pd.read_csv(io.StringIO(s),header=None,skiprows=4,sep=r'\s+',
                        names=['package','version','build','channel']).set_index('package')        
        for script in ['r','star','bedtools','samtools','deeptools','stringtie','scallop','kallisto','qualimap']:
            data[script+'_version'] = df.loc[script]['version']    
    with open(output_config_file,'w') as f:
        f.write(json.dumps(data,indent=4))
    print('# Wrote',output_config_file)
    if generate_design:
        rows = []
        fa_files = sorted([x for x in listdir(data['fastqFolder']) if x.endswith(data['extension'])])
        for fa_file in fa_files:
            sample = data['delim'].join(fa_file.replace('.'+data['extension'],'').split(data['delim'])[:-1])
            if design_condition_parser is not None:
                condition = design_condition_parser(fa_file)
            rows.append((sample,condition))                        
        df = pd.DataFrame(rows,columns=['sample','condition']).drop_duplicates(keep='first')
        df.to_csv(experimental_design,sep='\t',index=None)
        print('# Wrote',experimental_design)
    return data

In [4]:
conda_path = r'~/anaconda3/'
env_name = 'REAP'

conf = create_config_stub(output_config_file=output_config_file,fastq_path=output_fasta_path,
                   experimental_design=output_experimental_design,
                   annotation_gtf=annotation_gtf,reference_fasta=reference_fasta,
                   project_name='test_project',group='test_group',email='',
                   extension_fastq='fastq.gz',delim_fastq_name='_',suffix_mate1='r1',suffix_mate2='r2',
                    star_reference='',
                   conda_path=conda_path,env_name=env_name,
                    generate_design=True,design_condition_parser=(lambda x:x.split('_')[1]))

# Wrote /home/hugo/software/REAP/test_data/config.json
# Wrote /home/hugo/software/REAP/test_data/experimental_design.tsv
